# TP3
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 1

### Descrição do Problema


1. Pretende-se construir uma implementação simplificada do algoritmo “model checking” orientado aos interpolantes seguindo a estrutura apresentada nos apontamentos onde no passo $(n,m)\,$na impossibilidade de encontrar um interpolante invariante se dá ao utilizador a possibilidade de incrementar um dos índices $n$ e $m$ à sua escolha.
    Pretende-se aplicar este algoritmo ao problema da da multiplicação de inteiros positivos em `BitVec`  (apresentado no TP2).

### Abordagem do Problema

## Código Python

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import random as rn

In [ ]:
def geraBits(x,n):
    i = 0
    l = []
    while(x>0):
        i += 1
        l.insert(0,x%2)
        print(x)
        x = x//2
    if i>n:
        raise Exception("Não é possível representar o inteiro em " + str(n) + " bits!")
    while i < n:
        l.insert(0,0)
        i += 1
    return l

def listEquals(x,y,n):
    return And([Equals(x[i],y[i]) for i in range(n+1)])

def listNotEquals(x,y,n):
    return And([NotEquals(x[i],y[i]) for i in range(n+1)])

def listAdd(x,y,n): # soma x com o y e dá x como resultado
    return And([Equals(x[i],Plus(x[i],y[i])) for i in range(n+1)])

def listSub(x,y,n): # subtração entre x e y, dá x como resultado
    return And([Equals(x[i],Minus(x[i],y[i])) for i in range(n+1)])

#def listLS(x,times):
#    return x[times:] + ([0]*times)

#def listRS(x,times):
#    return ([0]*times) + x[:-times]

def listLS(x,times,n):
    if times > n+1 or times <= 0:
        raise Exception("erro")
    return And([Equals(x[i],x[i+1]) for i in range(n+1-times)]+[Equals(x[j],Int(0)) for j in range(n+1-times,n+1)])

def listRS(x,times,n):
    if times > n+1 or times <= 0:
        raise Exception("erro")
    return And([Equals(x[j],Int(0)) for j in range(times)]+[Equals(x[i],x[i-1]) for i in range(times,n+1)])


In [ ]:
def genState(vars,s,i,n):
    state = {}
    for v in vars:
        if v == 'pc':
            state[v] = Symbol(v+'!'+s+str(i),INT)
        else:
            state[v] = [Symbol(v+str(x)+'!'+s+str(i),INT) for x in range(n+1)]
    return state

In [ ]:
def init(state,a,b,n):
    if b > a:
        a,b = b,a
        
    tPc = Equals(state['pc'],Int(0)) # Program counter a zero
    tZ = listEquals(state['z'],geraBits(0,n+1),n) # Z a zero
    tX = listEquals(state['x'],geraBits(a,n+1),n) # x inicilizado com valor de a
    tY = listEquals(state['y'],geraBits(b,n+1),n) # y inicilizado com valor de b
    return And(tPc,tX,tY,tZ)

In [ ]:
def even(curr,prox,n):
    pcZero = Equals(curr['pc'],Int(0))
    yNotZero = listNotEquals(curr['y'],geraBits(0,n+1),n)
    yEven = Equals(curr['y'][n],Int(0))
    pc = Equals(prox['pc'],Int(1))
    x = Equals(prox['x'],curr['x'])
    y = Equals(prox['y'],curr['y'])
    z = Equals(prox['z'],curr['z'])
    return And(pcZero,yNotZero,yEven,pc,x,y,z)

def odd(curr,prox,n):
    pcZero = Equals(curr['pc'],Int(0))
    yNotZero = listNotEquals(curr['y'],geraBits(0,n+1),n)
    yOdd = Equals(curr['y'][n],Int(1))
    pc = Equals(prox['pc'],Int(2))
    x = Equals(prox['x'],curr['x'])
    y = Equals(prox['y'],curr['y'])
    z = Equals(prox['z'],curr['z'])
    return And(pcZero,yNotZero,yOdd,pc,x,y,z)

def backFromEven(curr,prox,n):
    pcOne = Equals(curr['pc'],Int(1))
    noError = Equals(curr['y'][0],Int(0))
    pc = Equals(prox['pc'],Int(0))
    x = Equals(prox['x'],listAdd(curr['x'],curr['x'],n))
    y = Equals(prox['y'],listRS(curr['y'],1,n))
    z = Equals(prox['z'],curr['z'])
    return And(pcOne,noError,pc,x,y,z)

def backFromOdd(curr,prox,n):
    pcTwo = Equals(curr['pc'],Int(2))
    noError = Equals(curr['y'][0],Int(0))
    pc = Equals(prox['pc'],Int(0))
    x = Equals(prox['x'],curr['x'])
    y = Equals(prox['y'],listSub(curr['y'],geraBits(1,n+1),n))
    z = Equals(prox['z'],listAdd(curr['z'],curr['x'],n))
    return And(pcOne,noError,yZero,pc,x,y,z)

def error(curr,prox,n):
    pcTwoOrThree = Or(Equals(curr['pc'],Int(2)),Equals(curr['pc'],Int(3)))
    yesError = Equals(curr['y'][0],Int(1))
    pc = Equals(prox['pc'],Int(4))
    x = Equals(prox['x'],curr['x'])
    y = Equals(prox['y'],curr['y'])
    z = Equals(prox['z'],curr['y'])
    return And(pcTwoOrThree,yesError,pc,x,y,z)

def endError(curr,prox,n)
    pcFour = Equals(curr['pc'],Int(4))
    pc = Equals(prox['pc'],Int(4))
    x = Equals(prox['x'],curr['x'])
    y = Equals(prox['y'],curr['y'])
    z = Equals(prox['z'],curr['y'])
    return And(pcFour,pc,x,y,z)

def stop(curr,prox,n):
    pcZero = Equals(curr['pc'],Int(0))
    yZero = listEquals(curr['y'],geraBits(0,n+1),n)
    pc = Equals(prox['pc'],Int(3))
    x = Equals(prox['x'],curr['x'])
    y = Equals(prox['y'],curr['y'])
    z = Equals(prox['z'],curr['z'])
    return And(pcZero,yZero,pc,x,y,z)

def end(curr,prox,n):
    pcThree = Equals(curr['pc'],Int(3))
    pc = Equals(prox['pc'],Int(3))
    x = Equals(prox['x'],curr['x'])
    y = Equals(prox['y'],curr['y'])
    z = Equals(prox['z'],curr['z'])
    return And(pcThree,pc,x,y,z)
    
    
def trans(curr,prox,n):    
    return Or(even(curr,prox,n),odd(curr,prox,n),backFromEven(curr,prox,n),
              backFromOdd(curr,prox,n),error(curr,prox,n),endError(curr,prox,n),
              stop(curr,prox,n),end(curr,prox,n))
